# ISO TC/211 Harmonised Model Controls

Connect to the EA app and model repository

In [1]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    omMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + omMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    closeEA(eaRepo)
    sys.exit()
printTS('Number of main packages: ' + str(omMod.Packages.Count))
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

2025-03-24 19:06:13   Hi EA - are you there? 
2025-03-24 19:06:13   I am here
2025-03-24 19:06:13   Hi EA - Please open this repository: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea
2025-03-24 19:06:15   OK! Repository C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea is ready!
2025-03-24 19:06:16   Model "Conceptual Models" found with PackageGUID {7B6B28E9-C583-4363-9E9C-F37A37AE06C9}
2025-03-24 19:06:16   Number of main packages: 5


## Select a package in EA before continuing

In [2]:
thePackage = eaRepo.GetTreeSelectedPackage()
printTS('Selected package name: ' + thePackage.Name)

2025-03-24 19:06:54   Selected package name: ISO/CD TS 19166 Edition 2


**Package tags and alias**

Add or update package tags and alias

In [3]:
lstMd = {}
lstMd['name'] = 'BIM to GIS conceptual mapping (B2GM)'
lstMd['number'] = '19166'
lstMd['edition'] = '2'
lstMd['publicationDate'] = ''
lstMd['yearVersion']= '2025'

eaEl = thePackage.Element

#Update existing tags and add missing tags
for key, value in lstMd.items():
    print(f"{key}: {value}")
    try:
        eaTag = eaEl.TaggedValues.GetByName(key)
        if not eaTag is None:
            eaTag.Value = value
            eaTag.Update()
        else:
            print('New tag!')
            eaTag = eaEl.TaggedValues.AddNew(key,value)
            eaTag.Update()    
    except:
        print('Something went wrong!')
eaEl.TaggedValues.Refresh()

#Set alias
eaEl.Alias = lstMd['name']
eaEl.Update()

#TODO:Update status?



name: BIM to GIS conceptual mapping (B2GM)
number: 19166
edition: 2
publicationDate: 
yearVersion: 2025


True

**Duplicates:**

Check that there are no duplicate element names

In [3]:
df = duplicateElements(thePackage)
non_unique = df[df.duplicated(subset=['ElementName'], keep=False)]
errCount = len(non_unique)
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Duplicate elements: ')
    for index, row in non_unique.iterrows():
        combined_string = f"{row['ElementName']}"
        print(combined_string)

2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: PK
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: B2GM
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: B2GM EM
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: B2GM LM
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: B2GM PD
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: BIM model
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: GIS model
2025-03-24 19:07:05   ERROR|Duplicate element: geometry
2025-03-24 19:07:05   ERROR|Duplicate element: property
2025-03-24 19:07:05   ERROR|Duplicate element: property_set
2025-03-24 19:07:05   ERROR|Duplicate element: relationship
2025-03-24 19:07:05   ERROR|Duplicate element: runtime
2025-03-24 19:07:05   ----------------------------
2025-03-24 19:07:05   Package: UC
20

**Data type references:**

Check that all data types are references to an element.
Missing references to primitive data types are fixed in the script.

The results are stored in a data frame for further use.

In [4]:
#df = pd.DataFrame(columns=['FullPath','Package','Element','Property','DependentPackage','DependentElement','GUID'])
df = listClassifiers(eaRepo,thePackage)
noRef = df[df['GUID'].isna()]
errCount = len(noRef)
printTS('')
printTS('Number of errors: ' + str(errCount))

print('')
print('Attributes without reference:')
for index, row in noRef.iterrows():
    combined_string = f"{row['Package']}.{row['Element']}.{row['Property']} (Data type:{row['DependentElement']})"
    print(combined_string)




2025-03-24 19:07:19   ----------------------------
2025-03-24 19:07:19   Package: PK
2025-03-24 19:07:20   ----------------------------
2025-03-24 19:07:20   Package: B2GM
2025-03-24 19:07:20   ----------------------------
2025-03-24 19:07:20   Package: B2GM EM
2025-03-24 19:07:20   ERROR|Missing data type connection for attribute :EM_rule.PSet_operation (Data type: {Replace, Append})
2025-03-24 19:07:20   ----------------------------
2025-03-24 19:07:20   Package: B2GM LM
2025-03-24 19:07:21   ----------------------------
2025-03-24 19:07:21   Package: B2GM PD
2025-03-24 19:07:21   ERROR|Missing data type connection for attribute :PD_property.type (Data type: {Integer, Real, CharacterString})
2025-03-24 19:07:23   ----------------------------
2025-03-24 19:07:23   Package: BIM model
2025-03-24 19:07:23   ERROR|Missing data type connection for attribute :property.type (Data type: {Integer, Real, CharacterString})
2025-03-24 19:07:24   ERROR|Missing data type connection for attribute :r

**Dependencies:**

Check which packages data types and relations are connected to.

In [17]:
dfCounts = df.groupby(['DependentPackage']).size()
print(dfCounts)

DependentPackage
ISO TC211.ISO 19103 Conceptual schema language.ISO 19103 Edition 2.Core Data Types                    29
ISO TC211.ISO 19107 Spatial schema.ISO 19107 Edition 2.Geometry                                        2
ISO TC211.ISO 19109 Rules for application schema.ISO/FDIS 19109 Edition 3.General Feature Model       10
ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.B2GM EM     6
ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.B2GM LM     8
ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.B2GM PD    19
ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.BIM model       17
ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.GIS model       14
dtype: int64


**Dependencies:**

List elements from a specific package 

In [15]:
strFilter = 'ISO TC211.ISO 19109 Rules for application schema.ISO 19109 Edition 2'

df_cleaned = df.dropna(subset=['DependentPackage'])
filtered_df = df_cleaned[df_cleaned['DependentPackage'].str.startswith(strFilter)]

if len(filtered_df) > 0:
    print('Refered elements from package "' + strFilter + '*"')
    for index, row in filtered_df.iterrows():
        print(f"{row['Element']}.{row['Property']} (Data type:{row['DependentPackage']}.{row['DependentElement']})")
else:
    print('No elements found!')    

No elements found!


**Upgrade model to new 19103**

Refer attribute data types to core types from 19103 Edition 2
Update classifiers to type and stereotype from 19103 Edition 2

In [5]:
dfCSL = fixCSL(eaRepo,thePackage)

2025-03-05 15:49:30   ----------------------------
2025-03-05 15:49:30   Package: PK
2025-03-05 15:49:30   ----------------------------
2025-03-05 15:49:30   Package: B2GM
2025-03-05 15:49:30   ----------------------------
2025-03-05 15:49:30   Package: B2GM EM
2025-03-05 15:49:30   INFO|Fixing referenced element for attribute: ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.CharacterString (GUID = {DDAA0BF3-9341-426c-8C73-FEAFDE87EB13})
2025-03-05 15:49:30   INFO|Fixing referenced element for attribute: ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.CharacterString (GUID = {DDAA0BF3-9341-426c-8C73-FEAFDE87EB13})
2025-03-05 15:49:30   INFO|Fixing referenced element for attribute: ISO TC211.ISO 19166 BIM to GIS conceptual mapping (B2GM).ISO/CD TS 19166 Edition 2.PK.B2GM.CharacterString (GUID = {DDAA0BF3-9341-426c-8C73-FEAFDE87EB13})
2025-03-05 15:49:30   INFO|Fixing referenced element for attribute: I

**Definitions:**

Check that all classes, attributes and navigable associations have definitions

In [5]:
#defDf = pd.DataFrame(columns=['Type','PackageName','ElementName','PropertyName','Supplier'])
defDf = listMissingDefinitions(eaRepo,thePackage)

errCount = len(defDf)
printTS('')
printTS('Number of errors: ' + str(errCount))
print('')


noDef = defDf[defDf['PropertyName'].isna() & defDf['Supplier'].isna()]
if len(noDef) > 0:
    print('')
    print('Elements without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Attribute']
if len(noDef) > 0:
    print('')
    print('Attributes without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Code value']
if len(noDef) > 0:
    print('')
    print('Code values without definitions (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without role name (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)

noDef = defDf[defDf['Type']=='Role']
if len(noDef) > 0:
    print('')
    print('Navigable association ends without definition (' + str(len(noDef)) + '):')
    for index, row in noDef.iterrows():
        if row['PropertyName'] != None:
            combined_string = f"{row['PackageName']}.{row['ElementName']}.{row['PropertyName']} towards {row['Supplier']}"
        else:
            combined_string = f"{row['PackageName']}.{row['ElementName']} towards {row['Supplier']}"
        print(combined_string)


2025-03-24 19:07:39   ----------------------------
2025-03-24 19:07:39   Package: PK
2025-03-24 19:07:39   ----------------------------
2025-03-24 19:07:39   Package: B2GM
2025-03-24 19:07:39   ----------------------------
2025-03-24 19:07:39   Package: B2GM EM
2025-03-24 19:07:39   ERROR|Missing definition for attribute in package B2GM EM: EM_destination.element
2025-03-24 19:07:39   ERROR|Missing definition for attribute in package B2GM EM: EM_source.element
2025-03-24 19:07:39   ERROR|Missing definition for attribute in package B2GM EM: EM_rule.name
2025-03-24 19:07:39   ERROR|Missing definition for attribute in package B2GM EM: EM_rule.PSet_operation
2025-03-24 19:07:39   ERROR|Missing role name for Navigable association in package B2GM EM: EM_rule towards EM_destination
2025-03-24 19:07:39   ERROR|Missing role definition for Navigable association in package B2GM EM: EM_rule towards EM_destination
2025-03-24 19:07:39   ERROR|Missing role name for Navigable association in package B2

**Diagram representation:**

Check that all elements are shown in at least one diagram

In [6]:
dfE = elementsInDiagrams(thePackage)
errCount = len(dfE)
printTS('')
printTS('Number of errors: ' + str(errCount))

if errCount > 0:
    print('')
    print('Elements that are not in any diagram:')
    for index, row in dfE.iterrows():
        combined_string = f"{row['PackageName']}.{row['ElementName']}"
        print(combined_string)    


2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: PK
2025-03-24 19:08:00   Diagram: PK (4 objects)
2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: B2GM
2025-03-24 19:08:00   Diagram: B2GM (3 objects)
2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: B2GM EM
2025-03-24 19:08:00   Diagram: B2GM EM (4 objects)
2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: B2GM LM
2025-03-24 19:08:00   Diagram: B2GM LM (4 objects)
2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: B2GM PD
2025-03-24 19:08:00   Diagram: B2GM PD (8 objects)
2025-03-24 19:08:00   ----------------------------
2025-03-24 19:08:00   Package: BIM model
2025-03-24 19:08:00   Diagram: BIM model (14 objects)
2025-03-24 19:08:01   ----------------------------
2025-03-24 19:08:01   Package: GIS model
2025-03-24 19:08:01   Diagram: GIS model (13 objects)
2025-03-24 19:08:01

In [ ]:
# Delete elements that are not in any diagram
for index, row in dfE.iterrows():
    combined_string = f"{row['PackageName']}.{row['ElementName']} ({row['GUID']})"
    #print(combined_string)    
    elGUID = row['GUID']
    el = eaRepo.GetElementByGuid(elGUID)
    #Delete the element
    if el:
        # Get the parent package of the element
        parent_package = eaRepo.GetPackageByID(el.PackageID)
        
        # Find and delete the element from the parent package
        for i in range(parent_package.Elements.Count):
            if parent_package.Elements.GetAt(i).ElementGUID == elGUID:
                parent_package.Elements.Delete(i)
                parent_package.Elements.Refresh()
                print(f"Element {elGUID} deleted successfully.")
                break
    else:
        print(f"Element {elGUID} not found.")

**Diagram layout:**

Setting diagram fonts to Cambria and hiding "isSubstitutable" labels

In [19]:
recDiagramCleaning(thePackage)

2025-03-17 12:01:50   Diagram count: 0
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Package: PK
2025-03-17 12:01:50   Diagram count: 1
2025-03-17 12:01:50   Diagram: PK (4 objects)
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Package: B2GM
2025-03-17 12:01:50   Diagram count: 1
2025-03-17 12:01:50   Diagram: B2GM (3 objects)
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Package: B2GM EM
2025-03-17 12:01:50   Diagram count: 1
2025-03-17 12:01:50   Diagram: B2GM EM (4 objects)
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Package: B2GM LM
2025-03-17 12:01:50   Diagram count: 1
2025-03-17 12:01:50   Diagram: B2GM LM (4 objects)
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Package: B2GM PD
2025-03-17 12:01:50   Diagram count: 1
2025-03-17 12:01:50   Diagram: B2GM PD (8 objects)
2025-03-17 12:01:50   ----------------------------
2025-03-17 12:01:50   Pack

Export error report

In [ ]:
print('')
file_path = mainFolder + '\\Controls.xlsx'
# Export to Excel 
writer = pd.ExcelWriter(file_path)
noDef = df[df['DependentPackage'].isna() | (df['DependentPackage'] == '')]
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing datatype references') 
    print(f"Exported datatype report to file: {file_path}") 

noDef = defDf[defDf['Type']!='Role name']
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing definitions') 
    print(f"Exported definitions report to file: {file_path}")   

noDef = defDf[defDf['Type']=='Role name']
if len(noDef) > 0:
    noDef.to_excel(writer,'Missing role names') 
    print(f"Exported role names report to file: {file_path}") 

if len(non_unique) > 0:
    non_unique.to_excel(writer,'Non unique names') 
    print(f"Exported duplication report to file: {file_path}") 

if len(dfE) > 0:
    dfE.to_excel(writer,'Not in diagram') 
    print(f"Exported diagram report to file: {file_path}")     

writer.close()



Exported datatype report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported definitions report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported role names report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx
Exported duplication report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Controls.xlsx


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_6384\1846826498.py:7: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing datatype references')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_6384\1846826498.py:12: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing definitions')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_6384\1846826498.py:17: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  noDef.to_excel(writer,'Missing role names')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_6384\1846826498.py:21: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  non_unique.to_excel(writer,'Non unique 

**Export to XMI:**

Export the package to XMI for upload to GitHub

In [ ]:
# Replace colons (":") with underscores ("_") in the filename
fName = thePackage.Name.replace(":", "_")
# Replace forward slashes ("/") with an empty string
fName = fName.replace("/", "")
# Combine the modified filename with the path and add the ".xml" extension
fName = xmiPath + fName + ".xml"

thePackage.IsControlled = -1
thePackage.XMLPath = fName
thePackage.BatchSave = 1
thePackage.BatchLoad = 1
thePackage.Update

#XmiExportType = 3
pI = eaRepo.GetProjectInterface()
result = pI.ExportPackageXMI(thePackage.PackageGUID, 3, 1, -1, 1, 0, fName)
print(result)